## Assignment 07 (60 pts) - Due December 4th, 11:59 PM

(5 pts) Be sure that your code is well-documented for reproducibility. Set a random seed at the top of your code for reproducibility (`np.random.seed(seed_number)`). Every function should have a docstring (simple functions can be short, longer functions should contain more detail). Before submitting, re-run your code. Figures should be labeled (axes labels, legends, etc), and clearly address any questions asked below! Remember, the whole goal is to have clear and concise communication of what you are finding computationally. 

Feel free to add cells as needed. You do not have to only use the cells that I provided.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

## Question 1
***
**Receptive field development with BCM rule**
Consider 20 presynaptic neurons with firing rates $r_j$ that connect onto the same postsynaptic neuron which fires at a rate $r^{post}=\sum_{j=1}^{20}w_jr_j$. Synaptic weights $w_j$ change according to the Bienenstock-Cooper-Munro (BCM) rule, with a hard lower bound $w_j\ge 0$ and $ r_\theta=10 $ Hz:

$$
    \frac{d}{dt}w_j = \eta r^{post}r_j(r^{post}-r_\theta)
$$

The 20 inputs are organized in two groups of 10 inputs each. There are two possible input patterns, $P^k$, $k=1,2$. Choose learning rate of $\eta=10^{-5}$. Initialize the weights as independently sampled from a uniform distribution between 0 and 1. Choose a time step size of $\Delta t = 1$.


**(5 pts)** **(a)** The two possible input patterns are: $P^1 $ -- group 1 fires at 3 Hz and group 2 is quiescent; and $P^2 $ --  group 1 is quiescent and group 2 fires at 0.5 Hz.  For each time step, randomly select input pattern from $P^1$ and $P^2$ with 50\% probability.   Set a upper bound of $w_j$ to be $w_{\rm max} =10$.  Simulate $N_t =1000$ time steps.  How do weights $w_{j}$  evolve? Plot a heatmap of the weights using `imshow` with x-axis being time and y-axis being the presynaptic neuron ID, $j$. Plot the  firing rates of the  post-synaptic neuron, $r^{\rm post} $, over time steps when the input patterns are $P^1$ or $P^2$, separately.   Show that the post-synaptic neuron becomes specialized to one group of inputs.

In [ ]:
### your code goes here ###

**(5 pts)** **(b)** Similar to (a), except that the second pattern now is  $k =2 $ -- group 1 is quiescent and group 2 fires at 2 Hz. How do weights $w_{j}$  and  firing rate $r^{\rm post} $ evolve? 

In [ ]:
### your code goes here ###

**(5 pts)** **(c)** As in (b), the two possible input patterns are: $P^1 $ -- group 1 fires at 3 Hz and group 2 is quiescent; and $P^2 $ --  group 1 is quiescent and group 2 fires at 2 Hz.  but make $r_\theta$ a function of the time-average firing rate $r^{\rm post} $ of the post-synaptic neuron. 
\begin{equation}
\tau_{\theta} \frac{dr_\theta}{dt} =  (r^{\rm post})^2/10 -r_\theta.  
\end{equation}
with $\tau_{\theta}  =20 $.  Simulate $N_t =5000$ time steps. Plot a heatmap of the presynaptic weights over time. Plot the  firing rates of the  post-synaptic neuron, $r^{\rm post}$, over time steps when the input patterns are $P^1$ or $P^2$, separately, and plot the threshold, $r_{\theta}$ over time.     Repeat a few times with different initialization of weights. 

In [ ]:
### your code goes here ###

***

# Question 2
***
**Perceptron binary classifier.**   Train a perceptron for binary classification. Each sample of input is  a $D$-dimensional vector,  $\vec{x}=(x_1, x_2, \ldots,  x_D)^T$. Each element $x_i$ is independent and normally distributed with standard deviation 1 and different mean.   Consider two categories of inputs.  Category 1 ($y=+1$): the mean of $\vec{x}$ is $\vec{c}_1$;  Category 2 ($y=-1$): the mean of $\vec{x}$ is $\vec{c}_2$ which is not equal to $\vec{c}_1$. Generate $K_{\rm train} = 10000$ training samples and  $K_{\rm test} = 1000$ test samples  for each group.   

The prediction of the perceptron classifier is   $\hat{y} = {\rm sign}(\sum_{i=1}^D w_i x_i +b)$, where  $\vec{w} = (w_1, w_2, \ldots,  w_D)^T$  is the connection weight vector and $b$ is the bias.

Initialize $\vec{w}$ and $b$ to be zeros. Randomize the order of training samples.  For each training sample $\vec{x}^{(k)}$, compute the output activity  $\hat{y} = {\rm sign}(\vec{w} \cdot \vec{x}^{(k)}  +b)$. Then update  $\vec{w}$ and $b$  using the perceptron learning rule: 

$$
\vec{w} \rightarrow \vec{w}+ \eta (y^{(k)} - \hat{y}) \vec{x}^{(k)} \\
b \rightarrow b+\eta (y^{(k)} - \hat{y})
$$

where $\eta = 0.001$ is the learning rate.  Iterate through all training samples, $k=1, 2, \ldots, K_{\rm train}$. 

**(5 pts)** **(a)** First choose $D=2$. Let $\vec{c}_1=[2,2]^T$ and  $\vec{c}_2=[-2, -2]^T$.  Plot the test samples $\vec{x}$ from category 1 in blue and the test samples from category 2 in red on the $x_1-x_2$ plane. After training, plot the decision boundary of the trained  linear classifier,  which is the line of $w_1 x_1 +w_2 x_2 +b = 0$.  

In [ ]:
### Initial Set up - Don't change this code ###
K = 10000
D = 2
center1 = 2 * np.ones((D, 1))
center2 = -2 * np.ones((D, 1))
x1 = np.random.randn(D, K) + center1
y1 = np.ones(K)
x2 = np.random.randn(D, K) + center2
y2 = -1 * np.ones(K)
X = np.concatenate([x1, x2], axis=1)
Y = np.concatenate([y1, y2])
permutes = np.random.permutation(2 * K)
X = X[:, permutes]
Y = Y[permutes]

Ktest = 1000
x1_test = np.random.randn(D, Ktest) + center1
y1_test = np.ones(Ktest)
x2_test = np.random.randn(D, Ktest) + center2
y2_test = -1 * np.ones(Ktest)
Xtest = np.concatenate([x1_test, x2_test], axis=1)
Ytest = np.concatenate([y1_test, y2_test])

In [ ]:
### your code goes here ###

**(5 pts)** **(b)** Compute the accuracy of the classifier on test samples, which is the percentage of samples where  the prediction $\hat{y}^{(k)}  = {\rm sign}(\vec{w} \cdot \vec{x}^{(k)} +b)$ equals the data category $y^{(k)}$.  

In [ ]:
### your code goes here ###

**(5 pts)** **(c)** Compare the final result of  $\vec{w}$ and the vector connecting the centers of inputs from the two categories,  $\vec{m} = \vec{c}_1 - \vec{c}_2$.  Compute the overlap between $\vec{w}$ and $\vec{m}$ as $p = \frac{\sum_{i=1}^D w_i m_i }{\sqrt{(\sum_{i=1}^D w_i^2)(\sum_{i=1}^D m_i^2)}}$.  

In [ ]:
### your code goes here ###


**(5 pts)** **(d)** Vary $D$ from 1 to 20 and let $\vec{c}_1=[0.5,0.5, \ldots, 0.5]^T$ and  $\vec{c}_2=[-0.5, -0.5,  \ldots, -0.5]^T$.  For each $D$, do 10 repetitions of the training process. Make a scatter plot of accuracy  and a scatter plot of the overlap between $\vec{w}$ and $\vec{m}$ across $D$ for all repetitions.  Show that the accuracy and overlap between $\vec{w}$ and $\vec{m}$ increase with the dimension of inputs.

In [ ]:
### your code goes here ###

***

# Question 3
***
**PCA on MNIST dataset.** The MNIST dataset is a large database of handwritten digits that is often used to benchmark machine learning algorithms. Each image is a 28x28 pixel grayscale imiage. For convenience, each 28x28 pixel image is unravelled into a single 784 (28^2) element vector. In the data file on Glow, `mnist.zip`, there is a training data set, trainX of 60,000 images and a testing data set testX of 10,000 images. Each row represents a different image, and each column represents a different pixel. 

You can visualize an image $k$ in the training set with `ax.imshow(trainX[k,:].reshape(28,28))`, an example is below. Each image has an associated label denoting which digit the image represents (0-9). The labels for the training images are saved in trainY, and the labes for the test images are saved in testY.

In [ ]:
### read in MNIST data ###
### Don't chage this code ###
### Assumes MNIST data in same directory as notebook ###
trainX = np.loadtxt("trainX.txt")
testX = np.loadtxt("testX.txt")
trainY = np.loadtxt("trainY.txt")
testY = np.loadtxt("testY.txt")
X = np.concatenate([trainX, testX])
n_samples, n_features = X.shape

# Example of plotting one image
plt.figure()
plt.imshow(X[378, :].reshape(28, 28), aspect="auto")
plt.axis("off")
plt.show()

**(5 pts)** **(a)**  Compute the mean of each column of $X$, $Xmean = mean(X)$ and subtract the mean from each column. Compute the covariance of $X$ using `np.cov` with the optional argument `rowvar=False` and eigenvalues of $\lambda_i$ of the covariance matrix. Determine the eigenvalues and eigenvectors with `np.linalg.eigh`. Plot all the eigenvalues sorted large to small (don't forget to rearrange the eigenvectors for later).

In [ ]:
### your code goes here ###

**(5 pts)** **(b)** The intrinsic dimensionality of the dataset can be studied via the fraction of variance explained by the top $d$ eigenvalues:

$$
\text{VarExplained}(d) = \frac{\sum_{i=1}^d \lambda_i}{\sum_{i=1}^N\lambda_i} \quad N = 784
$$

The intrinsic dimensionality is often quantified by the $d$ necessary to explain a large proportion of the total variance of the data (often a defined threshold, e.g. 90%). Plot VarExplained $(d)$ for $d=1$ to $d=784$. What is the minimum $d$ such that the threshold of 90% is reached?

In [ ]:
### your code goes here ###

**(5 pts)** **(c)** Let's reconstruct the data using only a subset of the principal components. Try with $D = 20$, $D= 80$, and $D = 200$. 

For each $D$, reconstruct $X$ with only $D$ dimensions. In other words, project onto the top $D$ principal compoents, $Y_D = X_0 V_D$ where $X_0$ is $X-mean(X)$ (from part (a)) and $V_D$ is the matrix of eigenvectors with $D$ dimensions. Then, find the reconstrcuted $X_{recon}$ by $X_{recon} = Y_D V_D^T$. Don't forget to add back the mean(X)!

Choose 5 example images and plot both the original and reconstrcuted images for all values of $D$. This will result in a 20 panel figure.

In [ ]:
### your code goes here ###

**(5 pts)** **(d)** Add Gaussian distributed independent noise to each pixel of the images $X$. Set the standard deviaion of the noise to be $0.2*max(X)$ and the mean of the noise to be 0. Repeat (a)-(c) from above with the noisy data. For part (a), you can skip the plot.

In [ ]:
### your code goes here ###

***